In [ ]:
""" Задачи """

Оценка разных статистик
  

Мы выяснили, что в эксперименте «Refactoring backend» в экспериментальной группе среднее время загрузки увеличилось, а 99% квантиль уменьшился. Проверьте значимость отличий других квантилей.

Данные эксперимента «Refactoring backend»: df_web_logs.csv и experiment_users.csv. Эксперимент проводился с 2022-04-05 по 2022-04-12. Измерения времени обработки запросов считаем независимыми. При проверке используйте нормальный доверительный интервал.

In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from scipy import stats

In [5]:
def read_file(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'Файл {file_name} не найден')
    return pd.read_csv(file_name)

In [9]:
df_web_logs = read_file('df_web_logs.csv')
df_users = read_file('experiment_users.csv')

In [43]:
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])

start_date = datetime(2022, 4, 5)
end_date = datetime(2022, 4, 12)

df_metrics = df_web_logs[(df_web_logs['date']>=start_date) 
                         & (df_web_logs['date']<end_date)][['user_id', 'load_time']].reset_index(drop=True)

df = pd.merge(df_users, df_metrics, on='user_id', how='left')[['pilot', 'load_time']]

In [44]:
df.head()

,pilot,load_time
0,0,106.6
1,0,49.6
2,0,49.9
3,0,75.7
4,0,61.6


In [ ]:
def get_ci_bootstrap_normal(boot_metrics: np.array, pe_metric: float, alpha: float=0.05):
    """Строит нормальный доверительный интервал.

    boot_metrics - значения метрики, полученные с помощью бутстрепа
    pe_metric - точечная оценка метрики
    alpha - уровень значимости
    
    return: (left, right) - границы доверительного интервала.
    """
    c = stats.norm.ppf(1 - alpha/2)
    se = np.std(boot_metrics)
    left, right = pe_metric - c*se, pe_metric + c*se
    return left, right

In [55]:
values_a = df[df['pilot']==0]['load_time'].to_numpy()
values_b = df[df['pilot']==1]['load_time'].to_numpy()